In [ ]:
from myosuite.utils import gym as myo_gym
from config import Config


In [13]:
cfg = Config()
env = myo_gym.make(cfg.env_id)


/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/glfw/__init__.py:917: GLFWError: (65550) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    


In [ ]:
action = env.action_space.sample()


In [18]:
_, _ = env.reset()
obs, base_reward, terminated, truncated, info = env.step(action)

In [25]:
info.keys()

dict_keys(['time', 'rwd_dense', 'rwd_sparse', 'solved', 'done', 'obs_dict', 'visual_dict', 'proprio_dict', 'rwd_dict', 'state'])

In [28]:
info['solved']

False

In [27]:
info['rwd_dict']

OrderedDict([('reach_dist', array(0.0628)),
             ('palm_dist', array(0.6068)),
             ('paddle_quat', array(0.3916)),
             ('torso_up', 0.9715726878623909),
             ('act_reg', array(-0.0162)),
             ('sparse', False),
             ('solved', array(False)),
             ('done', array(0)),
             ('dense', array(3.388))])